## Connections

In addition to `components`, users should acquaint themselves with another fundamental concept known as `connections`, which serves to establish links between two instances. JLNE supports an array of connection types such as one-to-one, one-to-many, many-to-one, and many-to-many where 'one' and 'many' pertain to the number of input sockets or output sockets.
> **Note**: JLNE does not currently allow for connecting multiple output sockets from a single instance to the same input socket, nor does it support connecting one output socket to several input sockets within the same instance.

### Add/Remove Connection

Please refer to the following animation to learn how to add/remove connections in JLNE<br>
![](./img/add_rm_con.gif)

### multi_connection

To distinguish between sockets that require a single connection and those that require multiple connections, we've introduced a parameter called `multi_connection` for both input and output sockets. Users can set this parameter to `True` if they need an input socket to accept multiple output sockets, or if they wish to connect a single output socket to multiple input sockets.

In [1]:
# import JLNE package
import jupyterlab_nodeeditor as jlne

# create a NodeEditor instance
ne = jlne.NodeEditor()

# specify the socket_types
coll = jlne.SocketCollection(socket_types = ('song', 'number', 'string'))

# define input ports
in1 = jlne.InputSlot(title = "Number 1", key = "num1", socket_type = "number", sockets = coll, multi_connection=True)
in2 = jlne.InputSlot(title = "Song 1", key = "song1", socket_type = "song", sockets = coll)
in3 = jlne.InputSlot(title = "String 1", key = "str1", socket_type = "string", sockets = coll)

# define output ports
out1 = jlne.OutputSlot(title = "Number 2", key = "num2", socket_type="number", sockets = coll, multi_connection=True)
out2 = jlne.OutputSlot(title = "Song 2", key = "song2", socket_type="song", sockets = coll)
out3 = jlne.OutputSlot(title = "String 2", key = "str2", socket_type="string", sockets = coll)

# define NumberInput Control
ctrl_num = jlne.NumberInputControlModel(key = 'num_key', editor = ne.node_editor)
ctrl_text = jlne.TextInputControlModel(key = 'text_key', editor = ne.node_editor, value="demonstration")
ctrl_dropdown = jlne.DropDownInputControlModel(key = 'dropdown_key', editor = ne.node_editor, options=[ {"text": "apple","value": "apple", },  
                                                                                                 {"text": "orange","value": "orange"},  
                                                                                                 {"text": "kiwi", "value": "orange"}])

# construct the component, which will show in the dropdown menu
comp_num = jlne.Component(title = "NumberInput", sockets=coll, inputs = [in1, in2, in3], outputs = [out1, out2, out3],
                   controls = [ctrl_num])

comp_text = jlne.Component(title = "TextInput", sockets=coll, inputs = [in1], outputs = [out1],
                   controls = [ctrl_text])

comp_dropdown = jlne.Component(title = "DropDownInput", sockets=coll, outputs = [out1, out2],
                   controls = [ctrl_dropdown])

# add component to the editor
ne.node_editor.add_component(comp_num)
ne.node_editor.add_component(comp_text)
ne.node_editor.add_component(comp_dropdown)

In this case, input socket `Number1` and output socket `Number2` can have multiple connnections

e.g., you can add three components to the workspace, connect 
- `DropDownInput Number2` and `NumberInput Number1`
- `TextInput Number2` and `NumberInput Number1`
- `DropDownInput Number2` and `TextInput Number1`

to understand this feature

In [2]:
ne

AppLayout(children=(Label(value='Node Editor', layout=Layout(grid_area='header')), Accordion(layout=Layout(gri…

### Add Connections by Scripts

Similar to add instances to workspace by scripts, we also implemented this approach for users to build connections by scripts

For instance, we tried to add connection to `DropDownInput Number2` and `NumberInput Number1`. First we need add instances to the workspace

**Add NodeInstance**

In [3]:
dropdown_instance=jlne.node_editor.NodeInstanceModel(
                                                    title = "DropDownInput",
                                                    sockets=coll, 
                                                    outputs = [out1, out2],
                                                    controls = [ctrl_dropdown]
                                                    )

In [4]:
number_instance=jlne.node_editor.NodeInstanceModel(title = "NumberInput", 
                                                   sockets= coll, 
                                                   inputs = [in1, in2, in3],
                                                   outputs = [out1, out2, out3],
                                                   controls = [ctrl_num]
                                                )

In [5]:
ne.node_editor.nodes = ne.node_editor.nodes + [dropdown_instance,number_instance]

**Add Connections**

The `source_node` refers to the `dropdown_instance`, and the `source_key` is the key representing the `Number2` socket. On the other hand, the `destination_node` stands for the `number_instance`, and the `destination_key` represents the key for the `Number1` socket.

In [6]:
conn = jlne.node_editor.ConnectionModel(
                    source_node=dropdown_instance,
                    source_key="num2",
                    destination_node=number_instance,
                    destination_key="num1",
                )

In [7]:
ne.node_editor.connections = ne.node_editor.connections + [conn]

Now, if you go back to the workspace, you will find out that the connection is already there